Copyright (c) 2024, NVIDIA CORPORATION.  All rights reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

MONAI Example adopted from https://github.com/Project-MONAI/tutorials/blob/main/2d_classification/monai_101.ipynb

Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

# MONAI 101 tutorial with Federated Learining

In this tutorial, we will introduce how simple it can be to run an end-to-end classification pipeline with MONAI.

These steps will be included in this tutorial, and each of them will take only a few lines of code:
- Dataset download
- Data pre-processing
- Define a DenseNet-121 and run training
- Check the results on test dataset

This tutorial will use about 7GB of GPU memory and 10 minutes to run.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Project-MONAI/tutorials/blob/main/2d_classification/monai_101.ipynb)

## Setup environment

In [1]:
!python -c "import monai" || pip install -q "monai-weekly[ignite, tqdm]"
!pip install nvflare

Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'monai'
  Using cached nvflare-2.4.0-py3-none-any.whl (2.1 MB)
  Using cached grpcio-1.51.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.8 MB)
  Using cached Flask_JWT_Extended-4.4.3-py2.py3-none-any.whl (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 KB 1.7 MB/s eta 0:00:003.0 MB/s eta 0:00:01
  Using cached websockets-12.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (130 kB)
  Using cached cryptography-42.0.5-cp39-abi3-manylinux_2_28_x86_64.whl (4.6 MB)
  Using cached Flask_SQLAlchemy-2.5.1-py2.py3-none-any.whl (17 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.1/385.1 KB 12.8 MB/s eta 0:00:00
  Using cached docker-7.0.0-py3-none-any.whl (147 kB)
  Using cached pyhocon-0.3.60-py3-none-any.whl
  Using cached Flask-2.2.5-py3-none-any.whl (101 kB)
  Using cached Werkzeug-2.2.2-py3-none-a

## Configure the NVFlare job templates folder

In [3]:
!nvflare config -jt ../../../../job_templates/
!nvflare job list_templates


The following job templates are available: 

------------------------------------------------------------------------------------------------------------------------
  name                 Description                                                  Controller Type   Execution API Type     
------------------------------------------------------------------------------------------------------------------------
  cyclic_cc_pt         client-controlled cyclic workflow with PyTorch ClientAPI tra client            NA                     
  cyclic_pt            server-controlled cyclic workflow with PyTorch ClientAPI tra server            NA                     
  psi_csv              private-set intersection for csv data                        server            NA                     
  sag_cross_np         scatter & gather and cross-site validation using numpy       server            NA                     
  sag_cse_pt           scatter & gather workflow and cross-site evaluation with Py

We will use the in-process client API, we choose the [sag_pt in_proc job template](../../../job_templates/sag_pt_in_proc) and run the following command to create the job:

In [4]:
!nvflare job create -force -j ./jobs/client_api -w sag_pt_in_proc -sd ./code/. \
    -f config_fed_client.conf app_script=monai_mednist_train.py
    -f config_fed_server.conf net_


The following are the variables you can change in the template

---------------------------------------------------------------------------------------------------------------------------------------
                                                                                                                                       
  job folder: ./jobs/client_api                                                                                                          
                                                                                                                                       
---------------------------------------------------------------------------------------------------------------------------------------
  file_name                      var_name                       value                               component                          
---------------------------------------------------------------------------------------------------------------------

Then we can run it using the NVFlare Simulator:

In [5]:
!nvflare simulator -n 2 -t 2 ./jobs/client_api -w client_api_workspace

2024-04-24 14:52:20,529 - SimulatorRunner - INFO - Create the Simulator Server.
2024-04-24 14:52:20,531 - CoreCell - INFO - server: creating listener on tcp://0:32795
2024-04-24 14:52:20,545 - CoreCell - INFO - server: created backbone external listener for tcp://0:32795
2024-04-24 14:52:20,546 - ConnectorManager - INFO - 2446133: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2024-04-24 14:52:20,546 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:11500] is starting
2024-04-24 14:52:21,047 - CoreCell - INFO - server: created backbone internal listener for tcp://localhost:11500
2024-04-24 14:52:21,048 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:32795] is starting
2024-04-24 14:52:21,123 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server localhost on Port 40957
2024-04-24 14:52:21,123 - SimulatorRunner - INFO - Deploy the Apps.
2024-04-24 14:52:21,134 - SimulatorRunner - INFO - Crea